# SPS Machine Learning Tutorial, Part 1
---
January 29/30, 2020 // Luc Le Pottier, University of Michigan


##### first: a cool viz which didn't fit last time
just because I didn't have time, here's another cool way of visualizing dimensional data.

Here we will show a cool method of transforming sound files (.wav), which are in the amplitude-time domain, to the frequency-time domain. 

First we get some .wav files: 

In [1]:
!wget -q https://github.com/luclepot/SPS_ml_tutorial/raw/master/data/synths/SYNTH-FancyPoly800Dazed.wav https://github.com/luclepot/SPS_ml_tutorial/raw/master/data/synths/SYNTH-PWMRezzzzzzz.wav
!ls *.wav

SYNTH-FancyPoly800Dazed.wav  SYNTH-PWMRezzzzzzz.wav


Then we can make what is called a spectrogram; basically, an image of the behavior of the .wav file in time, as a function of intensity.

In [ ]:
import numpy as np
import scipy.signal
import scipy.io.wavfile
import matplotlib.pyplot as plt 
import matplotlib.colors as plt_colors
import glob

filenames = glob.glob('SYNTH-*.wav')
for path in filenames:
    
    name = path.split('-')[-1].replace('.wav', '')
    sample_rate, data = scipy.io.wavfile.read(path)
    f,t,Sxx = scipy.signal.spectrogram(data, fs=sample_rate, return_onesided=True)

    plt.figure(figsize=(4*len(t)/len(f),4))
    plt.pcolormesh(t, f, Sxx, cmap='CMRmap', norm=plt_colors.LogNorm())
    plt.colorbar().set_label('intensity')

    plt.ylabel('frequency (hz)')
    plt.xlabel('time (s)')
    plt.gca().set_aspect(t.max()/f.max() * len(f)/len(t))
    plt.title(name)
    plt.show()


## 1. Building models

As I probably made exceedingly clear, building a machine learning model is in large part about getting good data. Therefore, lets grab some data first:

In [ ]:
!wget https://github.com/luclepot/SPS_ml_tutorial/raw/master/data/jets.h5 -q

Nice. We can load the data using h5, which we should have learned yesterday. Here's how to do it, but I encourage anyone to try it themselves first!

In [ ]:
import h5py

f = h5py.File('jets.h5', 'r')

print(list(f.keys()))

We can see there are a few datasets in this h5 file. We just want the `jet_features` column for now, which appears to be a group:

In [128]:
print(f['jet_features'])
print(list(f['jet_features'].keys()))

<HDF5 group "/jet_features" (2 members)>
['data', 'labels']


We can load the entirety of these datasets into memory because they are small, with the `labels` key being the columns of the set:

In [161]:
import pandas as pd

data_raw = f['jet_features']['data'][:] 
columns = f['jet_features']['labels'][:]
data_raw.shape, columns

((7406, 2, 9),
 array([b'Eta', b'Phi', b'Pt', b'M', b'ChargedFraction', b'PTD', b'Axis2',
        b'Flavor', b'Energy'], dtype='|S15'))

##### What is this data? 
Good question. This is data on individual jets at the LHC, with 9 attributes each per jet. 

##### What is a jet?
also a good question. Jets are analysis objects produced at LHC detectors.

the heavy and unstable results of high-energy particle collisions, i.e. the higgs boson or a top quark, decay rapidly after they are produced. So rapidly, in fact, that they do not make it anywhere close to the actual detectors!

Instead, these (and other) particles *shower*, or decay into various other hadrons (hadronization!). These hadrons then often decay as well, forming a cone of decay products. 

This cone of decay products - usually hadrons, electrons, and photons - is then intercepted by detectors, and it must be inferred what original particle generated these resultant particles. 

This is a *very* difficult problem, and an active field of research (my field of research!!)

##### What are we doing with this data?
These data points are from QCD (quark and gluon-produced) jets with very specific characteristics; namely, we have the leading and subleading (first and second highest transverse momentum, or $p_T$) jets for a number of events.

A good way to learn machine learning is to build a model. In this case, we can try to build a model to detect gluon jets Using the $P,~T,~E$ characterization from the lecture slides, we can define this as so:
* *$P$*: 

(13749, 6)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# reshape the data so it is all jets, and decode the feature labels
data = pd.DataFrame(data_raw.reshape((7406*2, 9)), columns=map(lambda x: x.decode('utf-8'), columns))
y = data['Flavor'].isin([9, 21]).astype(int)

data.drop(['Flavor', 'Eta', 'Phi'], axis=1, inplace=True)

plt.figure(figsize=(15,15))

data[y == 1].hist(label='gluons', color='tab:blue', histtype='step')
data[y == 0].hist(label='not gluons', color='tab:orange', histtype='step')
plt.legend()
plt.show()

##